# HIV Model
Gail Romer

In [85]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *


In [86]:
"""
Α α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, Ω ω
"""

'\nΑ α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, Ω ω\n'

In [132]:
# cd4_lymphocytes = 100;
# actively_effective_cells = 27;
# latently_affected_cells = 34;
# free_virons=263

cd4_lymphocytes = 200;
actively_effective_cells = 0;
latently_affected_cells = 0;
free_virons=100;



state = State(R = cd4_lymphocytes, L = latently_affected_cells, E = actively_effective_cells, V = free_virons)
system = System(Γ = 1.36,               #Per Day
                                            #constraint 1 of 'birthrate' of R
                τ = 0.2,                #Per Day
                                            #constraint 2 of 'birthrate' of R
                μ = 1.36/(10^-3),       #Per Day                             
                                            #deathrate of R and L (proportional to R and L themselves)
                Β = 0.00027,            #Per Viron
                                            #transitionrate from R to either E or L 
                    #Thats a beta, copy paste
                ρ = 0.1,                #Per Day
                                            #transitionrate of ΒR to L (proportional to free virons) (1-rho is transition rate of BR to E) 
                α = 3.6*(10^-2),        #Per Day
                                            #transitionrate of L to E
                δ = 0.33,               #Per Day
                                            #deathrate of E (proportional to E itself)
                π = 100,                #Per Day
                                            #'birthrate' of V (proportional to E) 
                σ = 2,                   #Per Day 
                                            #deathrate of V (proportional to V itself)
                t_0 = 0,
                
                t_end = 1000,
                
                dt = 0.01
               );

In [133]:
print(state)

R    200
L      0
E      0
V    100
dtype: int64


In [142]:
def update_func(state, t, system):
    """INFORMATION
    """
#     r = Γ*τ - μ*state.R - Β*state.R*state.V;
#     l = ρ*Β*state.R*state.V - μ*state.L - α*state.L;
#     e = (1-ρ)*Β*state.R*state.V + α*state.L - δ*state.E;
#     v = π*state.E - σ*state.V;
    
    R, L, E, V = state
    unpack(system)

# #     R = Γτ - μR - ΒRV;
#     r_production = Γ*τ - μ*R
#     r_elimination = Β*R*V
# #     L = ρΒRV - μL - αL;
#     l_production = ρ*Β*R*V
#     l_elimination = μ*L + α*L
# #     E = (1-ρ)ΒRV + αL - δE;
#     e_production = (1-ρ)*Β*R*V + α*L
#     e_elimination = δ*E 
# #     V = πE - σV;
#     v_production = π*E
#     v_elimination = σ*V
    
#     R = Γτ - μR - ΒRV;
    r_ = (Γ*τ - μ*R - Β*R*V) * dt
#     L = ρΒRV - μL - αL;
    l_ = (ρ*Β*R*V - μ*L - α*L) * dt
#     E = (1-ρ)ΒRV + αL - δE;
    e_ = ((1-ρ)*Β*R*V + α*L - δ*E) * dt
#     V = πE - σV;
    v_ = (π*E - σ*V) * dt
    
    
#     R += (r_production - r_elimination) * dt
#     L += (l_production - l_elimination) * dt 
#     E += (e_production - e_elimination) * dt
#     V += (v_production - v_elimination) * dt 

#     r = R + (r_production - r_elimination) * dt
#     l = L + (l_production - l_elimination) * dt 
#     e = E + (e_production - e_elimination) * dt
#     v = V + (v_production - v_elimination) * dt 

    r = R + r_
    l = L + l_ 
    e = E + e_
    v = V + v_
   
    
    return State(R=r, L=l, E=e, V=v)

In [143]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    unpack(system)
    
    frame = TimeFrame(columns=state.index)
    frame.row[t_0] = state
    
    for t in linrange(t_0, t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame


In [144]:
something = run_simulation(system, update_func)
# something.plot

C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in double_scalars
C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in double_scalars
C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars
C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


,R,L,E,V
0.0,200,0,0,100
1.0,200.251,0.0054,0.0486,98
2.0,200.503,0.0130396,0.0937946,96.0886
3.0,200.757,0.0238943,0.134669,94.2606
4.0,201.012,0.039362,0.169886,92.5101
5.0,201.268,0.0614467,0.197508,90.8298
6.0,201.526,0.0930204,0.214735,89.2107
7.0,201.784,0.1382,0.217528,87.6412
8.0,202.044,0.202886,0.200082,86.1059
9.0,202.305,0.295537,0.15405,84.5839


In [76]:
def slope_func(state, t, system):
    """INFORMATION
    """
    #     r = Γ*τ - μ*state.R - Β*state.R*state.V;
    #     l = ρ*Β*state.R*state.V - μ*state.L - α*state.L;
    #     e = (1-ρ)*Β*state.R*state.V + α*state.L - δ*state.E;
    #     v = π*state.E - σ*state.V;
    
    unpack(state)
    unpack(system)
    # #     R = Γτ - μR - ΒRV;
    #     r_production = Γ*τ - μ*R
    #     r_elimination = Β*R*V
    drdt = (Γ * τ - μ * R) - (Β * R * V)
    # #     L = ρΒRV - μL - αL;
    #     l_production = ρ*Β*R*V
    #     l_elimination = μ*L + α*L
    dldt = (ρ * Β * R * V) - (μ * L + α * L)
    # #     E = (1-ρ)ΒRV + αL - δE;
    #     e_production = (1-ρ)*Β*R*V + α*L
    #     e_elimination = δ*E 
    dedt = ((1-ρ) * Β * R * V + α * L) - (δ * E) 
    # #     V = πE - σV;
    #     v_production = π*E
    #     v_elimination = σ*V
    dvdt = (π * E) - (σ * V)
    
    return drdt, dldt, dedt, dvdt

In [77]:
slope_func(state,0,system)

(8.28211111111111, 1474.647877777778, -1471.3191000000002, 2174.0)

In [78]:
system.set(init=init, t_end=200)

In [79]:
results, details = run_ode_solver(system, slope_func)
details

TypeError: cannot convert dictionary update sequence element #0 to a sequence